In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

data_path = "faces/"
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]

Training_data, Labels = [], []
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)
Labels = np.asarray(Labels, dtype=np.int32)
model = cv2.face.LBPHFaceRecognizer_create()
model.train(np.asarray(Training_data), np.asarray(Labels))
print("model traind")

face_classifier = cv2.CascadeClassifier(
    "haarcascades/haarcascade_frontalface_default.xml"
)


def face_detector(img, size=0.5):
    grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(grey, 1.3, 3)
    if faces is ():
        return img, []

    for x, y, w, h in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 2)
        roi = img[y : y + h, x : x + w]
        roi = cv2.resize(roi, (200, 200))

    return img, roi


url = "http://172.20.10.6:8080"
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture(url + '/video')
while True:
    ret, frame = cap.read()

    img, face = face_detector(frame)
    # print(img)
    print(face)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        result = model.predict(face)
        if result[1] < 500:
            confidence = int(100 * (1 - (result[1]) / 300))
            display_string = str(confidence) + "% confidence"
        cv2.putText(
            img,
            display_string,
            (110, 120),
            cv2.FONT_HERSHEY_COMPLEX,
            1,
            (250, 120, 255),
        )

        if confidence > 75:
            cv2.putText(
                img, "Unlocked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0)
            )
            cv2.imshow("Face Cropper", img)
        else:
            cv2.putText(
                img, "locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255)
            )
            cv2.imshow("Face Cropper", img)

    except:
        cv2.putText(
            img, "No Face", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255)
        )
        cv2.imshow("Face Cropper", img)
        pass

    if cv2.waitKey(1) == 13:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
"""The provided code utilizes the cv2 (OpenCV) library in Python for face recognition. Here is a detailed description
of the code:

Importing the necessary modules:

The code imports cv2 for computer vision tasks and numpy for array manipulation.
It also imports modules from the standard library: listdir and isfile from the os module.
Setting up file paths and variables:

The variable data_path stores the path to the directory containing face images.
The onlyfiles list is populated with filenames of image files in the data_path directory.
Preparing training data:

The Training_data and Labels lists are initialized to store the face images and corresponding labels, respectively.
The code loops through the files and reads each image using cv2.imread().
The image is converted to grayscale using cv2.IMREAD_GRAYSCALE and appended to Training_data.
The label (index of the file) is appended to Labels.
Finally, the Labels list is converted to a NumPy array of integer type.
Training the model:

An instance of the cv2.face.LBPHFaceRecognizer model is created.
The model is trained using the face images in Training_data and their respective labels in Labels.
Defining the face detector:

The code defines a function called face_detector that takes an image and a size parameter.
The image is converted to grayscale.
The face_classifier cascade classifier is used to detect faces in the grayscale image.
Detected faces are drawn with rectangles on the image, and a region of interest (ROI) is extracted around each face.
The ROI is resized to a fixed dimension (200x200).
The function returns the modified image and the extracted ROI.
Capturing and processing video frames:

The code initializes a video capture object using either the webcam (cv2.VideoCapture(0)) or an IP camera stream.
The code enters a loop to continuously read frames from the video feed.
Each frame is passed to the face_detector function to detect faces and obtain the ROI.
The ROI is converted to grayscale and passed to the trained model for prediction.
The confidence level of the prediction is calculated and displayed on the frame.
Depending on the confidence level, the frame is annotated with "Unlocked" or "Locked" text indicating face recognition 
success or failure, respectively.
The annotated frame is displayed using cv2.imshow().
The loop continues until the user presses the Enter key.
Releasing resources:

Once the loop is terminated, the video capture object is released using cap.release().
All windows created by OpenCV are closed using cv2.destroyAllWindows().
This code provides a basic face recognition system that detects faces in video frames, compares them to a trained model, 
and displays the result in real-time. You can customize the code to suit your specific needs, such as using a different
face recognition algorithm or integrating it with other applications."""